# Libraries

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

#### Color Printing

In [2]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

#print(color.BOLD + 'Hello World !' + color.END)

# Ignore Deprecation & pandas warnings
Solution 1: This warning is caused by numpy which deprecated the truth value check on empty array
It is impossible to take advantage of the fact that empty arrays are False, because an array can be False for other reasons.


Solution 2: https://stackoverflow.com/questions/40033471/getting-settingwithcopywarning-warning-even-after-using-loc-in-pandas



In [28]:
import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)

pd.options.mode.chained_assignment = None


### Label Encoding

In [29]:
le = preprocessing.LabelEncoder()

# data_process(dataset_path) 
#### performs the data input & aggelos' processing.
# Preparexy(dataframe)
#### performs data cleaning & preparing.

In [30]:
def preparexy(df):
    # Data Preparing
    # Encode labels with value 0-> n_classes -1
    
    features = ['I50', 'Φ50', 'I150', 'Φ150', 'I250', 'Φ250']
    
    # Features - X
    X = df[features]

    # Calculate Z-score in order to find outliers
    z = np.abs(stats.zscore(X))
    #print(z) # Visualize
    threshold = 2.5 # Change the threshold arbitrarily
    #print(np.where(z > threshold))
    df = df[(z < threshold).all(axis=1)] # Remove outliers that exceed the threshold given from dataset

    # Now get as X the 'clean' features
    X = df[features]

    # Scale the inputs (4 options; put comment in the ones you are not using)

    # -1- Standard Scaling
    #scaler = StandardScaler()
    #X = scaler.fit_transform(X)

    # -2- Min-max scaling 
    #scaler = preprocessing.MinMaxScaler()
    #X = scaler.fit_transform(X)

    # -3- Robust scaling
    scaler = preprocessing.RobustScaler()
    X = scaler.fit_transform(X)

    # -4- Normalizing
    #scaler = preprocessing.Normalizer()
    #X = scaler.fit_transform(X)

    # Label - Y
    y = df['appliance']

    # Convert y to integer 
    le.fit(y) # Fit label encoder
    y = le.transform(y) # Transform labels to normalized encoding
    
    return(X,y)

def data_process(dataset_path):
    df = pd.read_excel(dataset_path)

    # Data Cleaning with Aggelos Rules for 50, 150, 250 phases
    df = df[(df.I50 > 0.1) & (df.I150 > 0.01) & (df.I250 > 0.01)] # Clean useless current features


    #df['Φ50'] = df['Φ50'].apply(pd.to_numeric)

    # For angle between (90, 180):
    # Modify by +180 degrees
    rows_with_rads_to_decrease = df.loc[(df['Φ50'] > 90) & (df['Φ50'] < 180)]
    rows_with_rads_to_decrease['Φ50'] -= 180
    df.update(rows_with_rads_to_decrease)

    # For angle between (-180, -90):
    # Modify by -180 degrees
    rows_with_rads_to_increase = (df.loc[(df['Φ50'] < -90) & (df['Φ50'] > -180)])
    rows_with_rads_to_increase['Φ50'] += 180
    df.update(rows_with_rads_to_increase)
    
    return df

### Load Datasets 
Following, you can either:
- train on house1, test on house2
- or split train/test set on houseX

In [31]:
# Uncomment the following if you want to train on house1 and test on house2:
#X_train, y_train = preparexy(data_process("../datasets/appliances_combination_daskio.xls"))
#X_test, y_test = preparexy(data_process("../datasets/appliances_combination_veroia.xls"))

######----#####------######----#####------######----#####------######----#####------

# Train & Testing in 1 dataset only
X, y = preparexy(data_process("../datasets/appliances_combination_daskio.xls"))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## printMetrics method (Used when splitting train/test so you have a $X_{test}$ and a $y_{test}$)
Takes as input the $classifier$, the $X_{test}$ and the $y_{test}$.

Then it prints a detailed report with all the known metrics. 

In [32]:
def printMetrics(clf, X_test, y_test):
    # Uncomment if you want confusion matrix to be shown
    #cm = confusion_matrix(y_test, y_pred) 
    #print("Confusion matrix")
    #print(cm)
    
    
    # This will print precision, recall, f1-score, support for all the categories
    #target_names = ['class 0', 'class 1', 'class 2']
    print("Classification report for classifier \n%s:\n%s" % (clf, metrics.classification_report(y_test, y_pred)))
    print("Accuracy: %1.3f" % clf.score(X_test, y_test))
    print("-----------------\n")   

# get_metrics Method
### Takes as input the true class and the predicted class
#### Then, splits these classes into the labels containing each class and computes Precision, Recall, Support

In [38]:
# Compute the metrics mathematically 
def get_metrics(true_string, predicted_string):
    
    true_array = true_string.split("+") # Find substrings and split them
    predicted_array = predicted_string.split("+")
    
    # Find which elements of array A are in array B
    comparePositives = np.in1d(predicted_array, true_array)
    compareNegatives = np.in1d(true_array, predicted_array)

    #print(comparePositives)
    #print(compareNegatives)
    
    TP = TN = FP = FN = 0 # Initialize the metrics
    
    # Scan the arrays for each elements' presence
    for predictedLabel in comparePositives:
        if (predictedLabel == True):
            TP = TP + 1
        if (predictedLabel == False):
            FP = FP + 1
            
    for predictedLabel in compareNegatives:
        if (predictedLabel == True):
            TN = TN + 1
        if (predictedLabel == False):
            FN = FN + 1
            

    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    #print(precision)
    #print(recall)
    
    if (precision+recall == 0):
        F1 = -1
    else:
        F1 = 2*((precision*recall)/(precision+recall))
    
    
    #print (precision, recall, F1)
    
    return precision, recall, F1

#print(get_metrics("Laptop+mpriki+Tostiera", "DVD-TV+Mati"))
#print(get_metrics("DVD-TV+Pistolaki+Tostiera", "DVD-TV+Pistolaki"))
#DVD-TV+Psigio+Mati 
#DVD-TV+Mati
#print(get_metrics("DVD-TV+Psigio+Mati", "DVD-TV+Mati"))

## Decision Trees
##### Uncomment and then add the for-loop (2 cells below) in order to run it

In [34]:
# Decision Trees
#clf = DecisionTreeClassifier(random_state = 42) # Feel free to change 'min_samples_split' 
#clf.fit(X_train, y_train)

#print("Decision Trees:")
#y_pred = clf.predict(X_test)

#printMetrics(clf, X_test, y_test)

## Multi-layer Perceptron with 18 neurons in 1 hidden layer

In [35]:
#print(X_test.shape, y_test.shape)
#print(le.inverse_transform(y_test[330])) # Check one TRUE class 

#true = le.inverse_transform(y_test[330])
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(18,), random_state=42)
#clf = LogisticRegression()
clf.fit(X_train, y_train)

print("Multi-layer Perceptron:")
y_pred = clf.predict(X_test)
#print(le.inverse_transform(y_pred[330])) # check one PREDICTED class
#predicted = le.inverse_transform(y_pred[330])

#printMetrics(clf, X_test, y_test)

Multi-layer Perceptron:


In [37]:
for i in range(0, len(y_test)):

    true_class = le.inverse_transform(y_test[i])
    predicted_class = le.inverse_transform(y_pred[i])
    print(true_class, predicted_class)
    
    #print(y_test[i], y_pred[i])
    #print('Actual class is {}{} and the classifier predicted {}'.format(color.BOLD, true_class, predicted_class))

    print('Actual class is ' + color.BOLD  + color.BLUE+ true_class + color.END +' and the classifier predicted ' + color.BOLD + color.RED + predicted_class + color.END)
    precision, recall, F1 = get_metrics(true_class, predicted_class)
    
    print('Precision is {}, Recall is {} and F1-Score is {}'.format(precision, recall, F1))
    
    print('\n')
    
    
    

DVD-TV+Laptop+Mati DVD-TV+Laptop+Mati
Actual class is DVD-TV+Laptop+Mati and the classifier predicted DVD-TV+Laptop+Mati
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Pistolaki+Mati DVD-TV+Pistolaki+Mati
Actual class is DVD-TV+Pistolaki+Mati and the classifier predicted DVD-TV+Pistolaki+Mati
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Laptop DVD-TV+Laptop
Actual class is DVD-TV+Laptop and the classifier predicted DVD-TV+Laptop
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


Laptop+mpriki+Tostiera Laptop+mpriki+Tostiera
Actual class is Laptop+mpriki+Tostiera and the classifier predicted Laptop+mpriki+Tostiera
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Psigio+Mati DVD-TV+Mati
Actual class is DVD-TV+Psigio+Mati and the classifier predicted DVD-TV+Mati
Precision is 1.0, Recall is 0.6666666666666666 and F1-Score is 0.8


Laptop+Psigio+Tostiera Laptop+Psigio+Tostiera
Actual class is Laptop+Psigio+Tostiera and the classifier predicted La

Laptop+Psigio+Mati Laptop+Psigio+Mati
Actual class is Laptop+Psigio+Mati and the classifier predicted Laptop+Psigio+Mati
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Psigio+Mati DVD-TV+Mati
Actual class is DVD-TV+Psigio+Mati and the classifier predicted DVD-TV+Mati
Precision is 1.0, Recall is 0.6666666666666666 and F1-Score is 0.8


DVD-TV+Laptop+Mati DVD-TV+Laptop+Mati
Actual class is DVD-TV+Laptop+Mati and the classifier predicted DVD-TV+Laptop+Mati
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Laptop+Psigio DVD-TV+Laptop+Psigio
Actual class is DVD-TV+Laptop+Psigio and the classifier predicted DVD-TV+Laptop+Psigio
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Psigio DVD-TV+Psigio
Actual class is DVD-TV+Psigio and the classifier predicted DVD-TV+Psigio
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Laptop DVD-TV+Laptop
Actual class is DVD-TV+Laptop and the classifier predicted DVD-TV+Laptop
Precision is 1.0, Recall is 1.0 and



Laptop+Pistolaki+Tostiera Laptop+Pistolaki+Tostiera
Actual class is Laptop+Pistolaki+Tostiera and the classifier predicted Laptop+Pistolaki+Tostiera
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Tostiera DVD-TV+Tostiera
Actual class is DVD-TV+Tostiera and the classifier predicted DVD-TV+Tostiera
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Psigio+Tostiera DVD-TV+Tostiera
Actual class is DVD-TV+Psigio+Tostiera and the classifier predicted DVD-TV+Tostiera
Precision is 1.0, Recall is 0.6666666666666666 and F1-Score is 0.8


DVD-TV+Pistolaki DVD-TV+Pistolaki
Actual class is DVD-TV+Pistolaki and the classifier predicted DVD-TV+Pistolaki
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Laptop+Mati DVD-TV+Laptop+Mati
Actual class is DVD-TV+Laptop+Mati and the classifier predicted DVD-TV+Laptop+Mati
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Laptop+Tostiera DVD-TV+Laptop+Tostiera
Actual class is DVD-TV+Laptop+Tostiera and the class

Precision is 0.6666666666666666, Recall is 0.6666666666666666 and F1-Score is 0.6666666666666666


DVD-TV+Mati DVD-TV+Psigio+Mati
Actual class is DVD-TV+Mati and the classifier predicted DVD-TV+Psigio+Mati
Precision is 0.6666666666666666, Recall is 1.0 and F1-Score is 0.8


Laptop+Psigio+Mati Laptop+Psigio+Mati
Actual class is Laptop+Psigio+Mati and the classifier predicted Laptop+Psigio+Mati
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Laptop+Tostiera DVD-TV+Laptop+Tostiera
Actual class is DVD-TV+Laptop+Tostiera and the classifier predicted DVD-TV+Laptop+Tostiera
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Laptop DVD-TV+Laptop
Actual class is DVD-TV+Laptop and the classifier predicted DVD-TV+Laptop
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Psigio+Mati DVD-TV+Psigio
Actual class is DVD-TV+Psigio+Mati and the classifier predicted DVD-TV+Psigio
Precision is 1.0, Recall is 0.6666666666666666 and F1-Score is 0.8


Pistolaki+mpriki+Tostier

Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Psigio+Mati DVD-TV+Mati
Actual class is DVD-TV+Psigio+Mati and the classifier predicted DVD-TV+Mati
Precision is 1.0, Recall is 0.6666666666666666 and F1-Score is 0.8


DVD-TV+Mati DVD-TV+Mati
Actual class is DVD-TV+Mati and the classifier predicted DVD-TV+Mati
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Laptop+Psigio DVD-TV+Psigio
Actual class is DVD-TV+Laptop+Psigio and the classifier predicted DVD-TV+Psigio
Precision is 1.0, Recall is 0.6666666666666666 and F1-Score is 0.8


Laptop+mpriki+Tostiera Laptop+mpriki+Tostiera
Actual class is Laptop+mpriki+Tostiera and the classifier predicted Laptop+mpriki+Tostiera
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Laptop+Tostiera DVD-TV+Laptop+Tostiera
Actual class is DVD-TV+Laptop+Tostiera and the classifier predicted DVD-TV+Laptop+Tostiera
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Mati Psigio+Pistolaki+Tostiera
Actual class is DVD

...

DVD-TV+Laptop+Mati DVD-TV+Laptop+Mati
Actual class is DVD-TV+Laptop+Mati and the classifier predicted DVD-TV+Laptop+Mati
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Laptop DVD-TV+Laptop
Actual class is DVD-TV+Laptop and the classifier predicted DVD-TV+Laptop
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Laptop+Pistolaki DVD-TV+Laptop+Pistolaki
Actual class is DVD-TV+Laptop+Pistolaki and the classifier predicted DVD-TV+Laptop+Pistolaki
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Laptop+Psigio DVD-TV+Laptop+Psigio
Actual class is DVD-TV+Laptop+Psigio and the classifier predicted DVD-TV+Laptop+Psigio
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


Laptop+mpriki+Tostiera Laptop+mpriki+Tostiera
Actual class is Laptop+mpriki+Tostiera and the classifier predicted Laptop+mpriki+Tostiera
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Mati DVD-TV+Mati
Actual class is DVD-TV+Mati and the classifier predicted DVD-TV+Mati
Precisio


DVD-TV+Laptop+Psigio DVD-TV+Laptop+Psigio
Actual class is DVD-TV+Laptop+Psigio and the classifier predicted DVD-TV+Laptop+Psigio
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


Laptop+Psigio+Pistolaki Laptop+Psigio+Pistolaki
Actual class is Laptop+Psigio+Pistolaki and the classifier predicted Laptop+Psigio+Pistolaki
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Pistolaki+Tostiera DVD-TV+Pistolaki
Actual class is DVD-TV+Pistolaki+Tostiera and the classifier predicted DVD-TV+Pistolaki
Precision is 1.0, Recall is 0.6666666666666666 and F1-Score is 0.8


DVD-TV+Pistolaki+Tostiera DVD-TV+Pistolaki+Tostiera
Actual class is DVD-TV+Pistolaki+Tostiera and the classifier predicted DVD-TV+Pistolaki+Tostiera
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Psigio DVD-TV+Tostiera
Actual class is DVD-TV+Psigio and the classifier predicted DVD-TV+Tostiera
Precision is 0.5, Recall is 0.5 and F1-Score is 0.5


DVD-TV+Psigio+Pistolaki DVD-TV+Psigio+Pistolaki
Actual clas

DVD-TV+Laptop+Mati DVD-TV+Laptop+Pistolaki
Actual class is DVD-TV+Laptop+Mati and the classifier predicted DVD-TV+Laptop+Pistolaki
Precision is 0.6666666666666666, Recall is 0.6666666666666666 and F1-Score is 0.6666666666666666


Laptop+Pistolaki Laptop+Pistolaki
Actual class is Laptop+Pistolaki and the classifier predicted Laptop+Pistolaki
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Laptop+Mati DVD-TV+Laptop+Mati
Actual class is DVD-TV+Laptop+Mati and the classifier predicted DVD-TV+Laptop+Mati
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Pistolaki+Tostiera DVD-TV+Pistolaki+Tostiera
Actual class is DVD-TV+Pistolaki+Tostiera and the classifier predicted DVD-TV+Pistolaki+Tostiera
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Tostiera DVD-TV+Tostiera
Actual class is DVD-TV+Tostiera and the classifier predicted DVD-TV+Tostiera
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Laptop+Psigio DVD-TV+Laptop+Psigio
Actual class is DVD-

Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Pistolaki+Tostiera DVD-TV+Pistolaki+Mati
Actual class is DVD-TV+Pistolaki+Tostiera and the classifier predicted DVD-TV+Pistolaki+Mati
Precision is 0.6666666666666666, Recall is 0.6666666666666666 and F1-Score is 0.6666666666666666


DVD-TV+Laptop+Mati DVD-TV+Laptop+Mati
Actual class is DVD-TV+Laptop+Mati and the classifier predicted DVD-TV+Laptop+Mati
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Laptop+Tostiera DVD-TV+Laptop+Tostiera
Actual class is DVD-TV+Laptop+Tostiera and the classifier predicted DVD-TV+Laptop+Tostiera
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Psigio+Pistolaki DVD-TV+Psigio+Tostiera
Actual class is DVD-TV+Psigio+Pistolaki and the classifier predicted DVD-TV+Psigio+Tostiera
Precision is 0.6666666666666666, Recall is 0.6666666666666666 and F1-Score is 0.6666666666666666


DVD-TV+Pistolaki+Mati DVD-TV+Pistolaki
Actual class is DVD-TV+Pistolaki+Mati and the classifier predic

Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Psigio DVD-TV+Psigio
Actual class is DVD-TV+Psigio and the classifier predicted DVD-TV+Psigio
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Laptop+Pistolaki DVD-TV+Laptop+Pistolaki
Actual class is DVD-TV+Laptop+Pistolaki and the classifier predicted DVD-TV+Laptop+Pistolaki
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Laptop DVD-TV+Laptop+Psigio
Actual class is DVD-TV+Laptop and the classifier predicted DVD-TV+Laptop+Psigio
Precision is 0.6666666666666666, Recall is 1.0 and F1-Score is 0.8


DVD-TV+Laptop+Psigio DVD-TV+Laptop+Psigio
Actual class is DVD-TV+Laptop+Psigio and the classifier predicted DVD-TV+Laptop+Psigio
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


Laptop+Psigio+Mati Laptop+Psigio+Mati
Actual class is Laptop+Psigio+Mati and the classifier predicted Laptop+Psigio+Mati
Precision is 1.0, Recall is 1.0 and F1-Score is 1.0


DVD-TV+Pistolaki DVD-TV+Pistolaki
Actual class is DVD-